## Microsoft Agent Framework 에이전트 유형

Microsoft 에이전트 프레임워크는 다양한 사용 사례 및 요구 사항을 충족하기 위해 여러 유형의 에이전트를 지원합니다.

모든 에이전트는 공통 기본 클래스인 AIAgent에서 파생되며, 이는 모든 에이전트 유형에 일관된 인터페이스를 제공합니다. 이를 통해 에이전트에 공통 기능, 에이전트만의 독자적 기능 뿐만 아니라 다중 에이전트 오케스트레이션과 같은 상위 수준 기능도 구축할 수 있습니다.

>**중요 사항**
>
>Microsoft Agent Framework를 사용하여 타사 서버 또는 에이전트와 연동되는 애플리케이션을 구축하는 경우, 그에 따른 위험은 전적으로 사용자에게 있습니다. 타사 서버 또는 에이전트와 공유하는 모든 데이터를 검토하고, 타사의 데이터 보존 및 위치 정책을 숙지하는 것이 좋습니다. 데이터가 조직의 Azure 규정 준수 및 지리적 경계를 벗어나 유출될 경우, 그로 인해 발생할 수 있는 모든 영향에 대한 책임은 사용자에게 있습니다.

### 추론(inference) 서비스를 기반으로 하는 간단한 에이전트

Agent Framework를 사용하면 다양한 추론 서비스를 기반으로 간단한 에이전트를 쉽게 만들 수 있습니다. 채팅 클라이언트 구현체를 제공하는 모든 추론 서비스를 사용하여 이러한 에이전트를 구축할 수 있습니다.

이러한 에이전트는 다음과 같은 다양한 기능을 기본적으로 지원합니다.

- 함수 호출(Function calling)
- 로컬 채팅 기록 또는 서비스에서 제공하는 채팅 기록 관리 기능을 활용한 다중-턴(Multi-turn) 대화
- 사용자 지정 서비스 제공 도구(예: MCP, 코드 실행)
- 구조화된 출력
- 스트리밍 응답

이러한 에이전트를 생성하려면, 원하는 채팅 클라이언트의 구현체를 사용하여 ChatAgent를 구성하면 됩니다.


In [3]:
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import DefaultAzureCredential

async with (
    DefaultAzureCredential() as credential,
    ChatAgent(
        chat_client=AzureAIAgentClient(credential=credential),
        instructions="You are a helpful assistant"
    ) as agent
):
    response = await agent.run("Hello!")
    print(response)

Hello! 👋 How can I help you today?


혹은, 다음과 같이 채팅 클라이언트 구현체에서 제공하는 간편한 메서드를 사용할 수도 있습니다.`


In [5]:
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import DefaultAzureCredential

async with DefaultAzureCredential() as credential:
    agent = AzureAIAgentClient(credential=credential).as_agent(
        instructions="You are a helpful assistant"
    )

    response = await agent.run("오늘도 힘내세요!")
    print(response)

감사합니다! 😊  
따뜻한 응원 덕분에 더욱 힘이 나는 것 같아요.  
오늘도 좋은 하루 보내시고, 하고 싶은 일 모두 잘 되길 바랍니다!  
필요한 게 있으면 언제든 말씀해 주세요. 힘내세요! 💪🌷


#### 함수 도구(Function Tools)

함수 도구를 사용하여 에이전트에 향상된 기능을 제공할 수 있습니다. 자세한 실습은 이어지는 과정에서 다룹니다.

In [8]:
from typing import Annotated
from pydantic import Field
from azure.identity.aio import DefaultAzureCredential
from agent_framework.azure import AzureAIAgentClient

def get_weather(location: Annotated[str, Field(description="The location to get the weather for.")]) -> str:
    """Get the weather for a given location."""
    return f"The weather in {location} is sunny with a high of 25°C."

async with (
    DefaultAzureCredential() as credential,
    AzureAIAgentClient(credential=credential).as_agent(
        instructions="You are a helpful weather assistant.",
        tools=get_weather
    ) as agent
):
    response = await agent.run("What's the weather in Seattle?")
    print(response.text)

The weather in Seattle is currently sunny with a high of 25°C.


#### 응답 스트리밍(Streaming Responses)

에이전트는 일반적인 응답과 스트리밍 응답을 모두 지원합니다. 스트리밍 응답을 사용하면 에이전트가 생성하는 콘텐츠를 점진적으로 수신할 수 있어, 대기 시간을 줄이고 사용자 경험을 향상시킬 수 있습니다. 자세한 실습은 이어지는 과정에서 다룹니다.

In [11]:
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import DefaultAzureCredential

async with DefaultAzureCredential() as credential:
    agent = AzureAIAgentClient(credential=credential).as_agent(
        instructions="You are a helpful assistant"
    )

    # Streaming response (get results as they are generated)
    async for chunk in agent.run_stream("가장 인기 있는 SUV 차량은 무엇인가요?"):
        if chunk.text:
            print(chunk.text, end="", flush=True)

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x00000208FA136BD0>


현재(2024년 기준) 전 세계적으로 가장 인기 있는 SUV 차량은 여러 가지가 있지만, 판매량과 인기도를 기준으로 아래 몇 가지 차종이 대표적입니다:

### 1. **현대 투싼(Hyundai Tucson)**
- 2023~2024년에도 글로벌 및 국내 시장에서 매우 높은 판매량을 기록.
- 합리적인 가격, 좋은 연비, 첨단 편의사양 덕분에 꾸준히 인기.

### 2. **기아 스포티지(Kia Sportage)**
- 글로벌 판매 1~2위를 다투는 차량.
- 신형 모델 출시로 디자인과 기능 모두 호평.

### 3. **도요타 RAV4(Toyota RAV4)**
- 미국, 유럽 등 해외 시장에서 단연 인기.
- 하이브리드 모델 등 친환경 옵션도 인기 요인.

### 4. **혼다 CR-V(Honda CR-V)**
- 오랜 기간 세계적으로 베스트셀러 SUV 중 하나.
- 실용성과 내구성에서 강점.

### 5. **테슬라 모델 Y(Tesla Model Y)**
- 순수 전기 SUV 중 세계 1~2위 판매량.
- 친환경, 전기차 시장에서 매우 높은 성장세.

### 6. **제네시스 GV70**
- 프리미엄 SUV 시장에서 큰 성공.
- 2022~2024년 꾸준히 많은 수상 경력과 판매 상승.

국내 기준으로 가장 인기 있는 SUV는 "현대 투싼", "기아 스포티지", "제네시스 GV70" 등이 꼽히고, 세계 판매량 기준으로는 "도요타 RAV4", "테슬라 모델 Y", "혼다 CR-V" 등이 가장 인기 있습니다.

**원하는 시장(한국, 미국, 유럽 등)이나 SUV 등급(준중형, 중형, 대형 등)에 따라 상세 추천도 가능합니다. 구체적으로 어떤 용도를 원하시는지 알려주시면 더 맞춤형 정보를 드릴 수 있습니다!**

#### 코드 인터프리터 도구(Code Interpreter Tools)

Azure AI 에이전트는 코드 인터프리터 도구를 내부적으로 호스팅하며 지원하기에 Python 코드를 실행할 수 있습니다.

In [20]:
from agent_framework import ChatAgent, HostedCodeInterpreterTool
from agent_framework.azure import AzureAIAgentClient, AzureOpenAIChatClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
async with ChatAgent(
    chat_client=AzureAIAgentClient(credential=credential),
    instructions="You are a helpful assistant that can execute Python code.",
    tools=HostedCodeInterpreterTool()
) as agent:

    response = await agent.run("Calculate the factorial of 100 using Python")
    print(response)

The factorial of 100 (100!) is:

```
93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000
```


또한, MAF에는 다양한 유형의 에이전트가 지원됩니다. Azure AI Foundry Agent, Azure OpenAI Agent, OpenAI Agent, Anthropic Agents, GitHub Copilot Agent 등이 있습니다. 이들은 각 유형에 따라 다양한 기능을 지원합니다. 예를 들어, 일부 에이전트는 서비스에서 제공하는 채팅 기록 저장소를 지원하는 반면, 다른 에이전트는 사용자 지정 채팅 기록 저장소를 지원합니다. 또한, IChatClient 인터페이스를 구현하는 모든 채팅 클라이언트를 기반으로 하는 사용자 지정 에이전트도 만들 수 있습니다. 

자세한 정보는 공식 사이트의 [지원되는 에이전트 유형](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/agent-types) 문서를 참조하세요.

#### 사용자 지정 에이전트(Custom agents)

원한다면, 채팅 클라이언트를 단순히 래핑하는 것이 아닌 완전한 사용자 지정 에이전트를 만들 수도 있습니다. 에이전트 프레임워크는 AgentProtocol 프로토콜과 BaseAgent 기본 클래스를 제공하기에, 이를 구현/하위 클래스화하면 에이전트의 동작과 기능을 완전히 제어할 수 있습니다. 다음은 코드 예시입니다.


In [ ]:
from typing import Any
from agent_framework import BaseAgent, AgentResponse, AgentResponseUpdate, AgentThread, ChatMessage
from collections.abc import AsyncIterable

class CustomAgent(BaseAgent):
    async def run(
        self,
        messages: str | ChatMessage | list[str] | list[ChatMessage] | None = None,
        *,
        thread: AgentThread | None = None,
        **kwargs: Any,
    ) -> AgentResponse:
        # Custom agent implementation
        pass

    def run_stream(
        self,
        messages: str | ChatMessage | list[str] | list[ChatMessage] | None = None,
        *,
        thread: AgentThread | None = None,
        **kwargs: Any,
    ) -> AsyncIterable[AgentResponseUpdate]:
        # Custom streaming implementation
        pass

NameError: name 'Any' is not defined